# Model DB

In [1]:
import pandas as pd 
import wandb
from pathlib import Path

from workbench.wandb import wandb_model_DB, make_clickable, get_architecture_from_model_DB_run_id, get_model_DB_run_id_from_architecture


%load_ext autoreload

In [2]:
%reload_ext autoreload
%autoreload

In [3]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# Get model_DB

In [29]:
df = wandb_model_DB()
df.head()

,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,tags,created_at,path,notes,test_accuracy
0,f7cli26n,mobilenetv2_0.2_96_c3_o2_t3l128.MV1,finished,1.881 M,https://wandb.ai/susbrock/model_DB/runs/f7cli26n,mobilenetv2_0.2_96_c3_o2_t3l128.MV1,4.024 M,0.2,2,96,4169783,3,1881445,t3l128.MV1,0.003507,55730,4280383,674.406250,55296.0,1881445,0.003707,52094,3636,220.859375,226160,0,130.843750,133984,209928,0,81636,[],2023-03-01T19:35:25,"[susbrock, model_DB, f7cli26n]",None,None
1,le3i8f7h,mobilenetv2_0.2_96_c3_o2_t3l256.MV1,finished,1.955 M,https://wandb.ai/susbrock/model_DB/runs/le3i8f7h,mobilenetv2_0.2_96_c3_o2_t3l256.MV1,4.176 M,0.2,2,96,4322487,3,1955429,t3l256.MV1,0.003536,64690,4433087,710.429688,55296.0,1955429,0.003735,60798,3892,254.359375,260464,0,142.593750,146016,210952,0,82660,[],2023-03-01T19:33:21,"[susbrock, model_DB, le3i8f7h]",None,None
2,phil00x9,mobilenetv2_0.2_96_c3_o2_t3l512.MV1,finished,2.103 M,https://wandb.ai/susbrock/model_DB/runs/phil00x9,mobilenetv2_0.2_96_c3_o2_t3l512.MV1,4.479 M,0.2,2,96,4627895,3,2103397,t3l512.MV1,0.003581,82610,4738495,780.484375,55296.0,2103397,0.003781,78206,4404,321.359375,329072,0,166.093750,170080,213000,0,84708,[],2023-03-01T19:31:18,"[susbrock, model_DB, phil00x9]",None,None
3,pksk1h3b,mobilenetv2_0.2_96_c3_o2_t3l1024.MV1,finished,2.399 M,https://wandb.ai/susbrock/model_DB/runs/pksk1h3b,mobilenetv2_0.2_96_c3_o2_t3l1024.MV1,5.085 M,0.2,2,96,5238711,3,2399333,t3l1024.MV1,0.003571,118450,5349311,920.484375,55296.0,2399333,0.003771,113022,5428,455.359375,466288,0,213.093750,218208,217096,0,88804,[],2023-03-01T19:29:17,"[susbrock, model_DB, pksk1h3b]",None,None
4,a2wlivv7,mobilenetv2_0.2_96_c3_o2_t4l1024.MV1,finished,2.762 M,https://wandb.ai/susbrock/model_DB/runs/a2wlivv7,mobilenetv2_0.2_96_c3_o2_t4l1024.MV1,5.870 M,0.2,2,96,6052770,3,2762204,t4l1024.MV1,0.004026,133522,6163370,977.968750,55296.0,2762204,0.004225,127174,6348,507.332031,519508,0,237.046875,242736,220424,0,92160,[],2023-03-01T19:27:13,"[susbrock, model_DB, a2wlivv7]",None,None


In [46]:
df.test_accuracy = df.test_accuracy.fillna(0)
df.test_accuracy.replace("NaN", 0, inplace=True)

In [47]:
df.test_accuracy.unique()

array([0.        , 0.83216274, 0.83690596, 0.81574386, 0.81430137,
       0.80598378, 0.7487914 , 0.76302105, 0.83343977, 0.83353096,
       0.87293625, 0.79987228, 0.77123052, 0.85186535, 0.85113561,
       0.82869655, 0.81282496, 0.85077077, 0.83891273, 0.80187905,
       0.82577759])

# Get training info
just export run_table from wandb

In [30]:
#def wandb_model_DB():
api = wandb.Api()
runs = api.runs("susbrock/model_DB_visual_wake_words")

config_df = pd.DataFrame()
name = []
tags =[]
state = []
url = []
created_at = []
path = []
notes = []
test_accuracies = []
#test_losses = []

for run in runs:
    name.append(run.name)
    tags.append(run.tags)
    state.append(run.state)
    url.append(run.url)
    created_at.append(run.created_at)
    path.append(run.path)
    notes.append(run.notes)
    test_accuracies.append(run.summary.get("test_accuracy"))

    run_df = pd.DataFrame([run.config], columns=run.config.keys())
    config_df = pd.concat([config_df, run_df], axis=0, ignore_index=True)
col_names = ["run_name", "tags", "state", "url", "created_at", "path", "notes", "test_accuracy"]
attrs_df = pd.DataFrame(zip(name, tags, state, url,created_at, path, notes, test_accuracies), columns=col_names)

training_df =  pd.concat([config_df, attrs_df], axis=1)

training_df.insert(1,"run_name" , training_df.pop("run_name"))
training_df.insert(3,"architecture" , training_df.pop("architecture"))   
training_df.insert(2,"state" , training_df.pop("state")) 
training_df.insert(4,"url" , training_df.pop("url")) 



In [31]:
training_df.head()

,id,run_name,state,seed,url,architecture,epochs,classes,dataset,momentum,optimizer,batch_size,learn_rate,batch_norm_momentum,hostname,tags,created_at,path,notes,test_accuracy
0,tuq3r964,mobilenetv2_0.2_96_c3_o2_t4l256.MV1,running,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t4l256.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T19:09:57,"[susbrock, model_DB_visual_wake_words, tuq3r964]",None,NaN
1,0400umu4,mobilenetv2_0.2_96_c3_o2_t4l256.MV1,finished,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t4l256.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T18:27:15,"[susbrock, model_DB_visual_wake_words, 0400umu4]",None,NaN
2,wh6k8s0w,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,finished,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T18:20:18,"[susbrock, model_DB_visual_wake_words, wh6k8s0w]",None,NaN
3,bq8c0lhi,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,finished,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T18:13:08,"[susbrock, model_DB_visual_wake_words, bq8c0lhi]",None,NaN
4,3x1qra2a,shufflenetv2tiny_0.2_96_c3_o2_f24l1024,crashed,15,https://wandb.ai/susbrock/model_DB_visual_wake...,shufflenetv2tiny_0.2_96_c3_o2_f24l1024,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T14:43:04,"[susbrock, model_DB_visual_wake_words, 3x1qra2a]",None,NaN


In [32]:
df.columns

Index(['id', 'run_name', 'state', 'MACs', 'url', 'architecture', 'FLOPs',
       'alpha', 'classes', 'img_res', 'OPS_mltk', 'channels', 'macs_mltk',
       'variation', 'energy_mltk', 'total_params', 'OPS_INT8_mltk',
       'model_size_kb', 'peak_memory_b', 'macs_INT8_mltk', 'energy_INT8_mltk',
       'trainable_params', 'non_trainable_params', 'tflite_model_size_kb',
       'flash_model_size_b_mltk', 'n_unsupported_layers_mltk',
       'tflite_model_INT8_size_kb', 'flash_model_size_b__INT8_mltk',
       'RAM_runtime_memory_size_b_mltk', 'n_unsupported_layers_INT8_mltk',
       'RAM_runtime_memory_size_b_INT8_mltk', 'tags', 'created_at', 'path',
       'notes', 'test_accuracy'],
      dtype='object')

In [60]:
def get_untrained_models():
    df = wandb_model_DB()
    wandb_untrained_df = df[df["test_accuracy"]<0.5]
    architectures = wandb_untrained_df["architecture"].unique().tolist()
    return architectures

In [49]:
wandb_untrained_df = df[df["test_accuracy"]<0.5]  #.test_accuracy==0].copy
wandb_untrained_df

,id,run_name,state,MACs,url,architecture,FLOPs,alpha,classes,img_res,OPS_mltk,channels,macs_mltk,variation,energy_mltk,total_params,OPS_INT8_mltk,model_size_kb,peak_memory_b,macs_INT8_mltk,energy_INT8_mltk,trainable_params,non_trainable_params,tflite_model_size_kb,flash_model_size_b_mltk,n_unsupported_layers_mltk,tflite_model_INT8_size_kb,flash_model_size_b__INT8_mltk,RAM_runtime_memory_size_b_mltk,n_unsupported_layers_INT8_mltk,RAM_runtime_memory_size_b_INT8_mltk,tags,created_at,path,notes,test_accuracy
0,f7cli26n,mobilenetv2_0.2_96_c3_o2_t3l128.MV1,finished,1.881 M,https://wandb.ai/susbrock/model_DB/runs/f7cli26n,mobilenetv2_0.2_96_c3_o2_t3l128.MV1,4.024 M,0.20,2,96,4169783,3,1881445,t3l128.MV1,0.003507,55730,4280383,674.406250,55296.0,1881445,0.003707,52094,3636,220.859375,226160,0,130.843750,133984,209928,0,81636,[],2023-03-01T19:35:25,"[susbrock, model_DB, f7cli26n]",None,0.0
1,le3i8f7h,mobilenetv2_0.2_96_c3_o2_t3l256.MV1,finished,1.955 M,https://wandb.ai/susbrock/model_DB/runs/le3i8f7h,mobilenetv2_0.2_96_c3_o2_t3l256.MV1,4.176 M,0.20,2,96,4322487,3,1955429,t3l256.MV1,0.003536,64690,4433087,710.429688,55296.0,1955429,0.003735,60798,3892,254.359375,260464,0,142.593750,146016,210952,0,82660,[],2023-03-01T19:33:21,"[susbrock, model_DB, le3i8f7h]",None,0.0
2,phil00x9,mobilenetv2_0.2_96_c3_o2_t3l512.MV1,finished,2.103 M,https://wandb.ai/susbrock/model_DB/runs/phil00x9,mobilenetv2_0.2_96_c3_o2_t3l512.MV1,4.479 M,0.20,2,96,4627895,3,2103397,t3l512.MV1,0.003581,82610,4738495,780.484375,55296.0,2103397,0.003781,78206,4404,321.359375,329072,0,166.093750,170080,213000,0,84708,[],2023-03-01T19:31:18,"[susbrock, model_DB, phil00x9]",None,0.0
3,pksk1h3b,mobilenetv2_0.2_96_c3_o2_t3l1024.MV1,finished,2.399 M,https://wandb.ai/susbrock/model_DB/runs/pksk1h3b,mobilenetv2_0.2_96_c3_o2_t3l1024.MV1,5.085 M,0.20,2,96,5238711,3,2399333,t3l1024.MV1,0.003571,118450,5349311,920.484375,55296.0,2399333,0.003771,113022,5428,455.359375,466288,0,213.093750,218208,217096,0,88804,[],2023-03-01T19:29:17,"[susbrock, model_DB, pksk1h3b]",None,0.0
4,a2wlivv7,mobilenetv2_0.2_96_c3_o2_t4l1024.MV1,finished,2.762 M,https://wandb.ai/susbrock/model_DB/runs/a2wlivv7,mobilenetv2_0.2_96_c3_o2_t4l1024.MV1,5.870 M,0.20,2,96,6052770,3,2762204,t4l1024.MV1,0.004026,133522,6163370,977.968750,55296.0,2762204,0.004225,127174,6348,507.332031,519508,0,237.046875,242736,220424,0,92160,[],2023-03-01T19:27:13,"[susbrock, model_DB, a2wlivv7]",None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,pnq7ad7c,mobilenetv1_0.35_96_c3_o2_l1,finished,8.950 M,https://wandb.ai/susbrock/model_DB/runs/pnq7ad7c,mobilenetv1_0.35_96_c3_o2_l1,18.706 M,0.35,2,96,18705937,3,8949947,l1,0.007999,277291,18816537,1291.246094,76032.0,8949947,0.008198,272515,4776,1053.855469,1079148,0,336.125000,344192,429332,0,125652,[],2023-02-05T21:01:06,"[susbrock, model_DB, pnq7ad7c]",None,0.0
86,i4nnlhl8,mobilenetv1_0.25_96_c3_o2_l5,finished,7.490 M,https://wandb.ai/susbrock/model_DB/runs/i4nnlhl8,mobilenetv1_0.25_96_c3_o2_l5,15.674 M,0.25,2,96,15673996,3,7489664,l5,0.005974,221794,15784596,1123.593750,55296.0,7489664,0.006173,216322,5472,834.507812,854536,0,294.171875,301232,322644,0,101956,[],2023-02-05T20:59:40,"[susbrock, model_DB, i4nnlhl8]",None,0.0
87,ylfo6y2p,mobilenetv1_0.25_96_c3_o2_l4,finished,6.858 M,https://wandb.ai/susbrock/model_DB/runs/ylfo6y2p,mobilenetv1_0.25_96_c3_o2_l4,14.384 M,0.25,2,96,14383756,3,6858368,l4,0.005664,202978,14494356,1034.257812,55296.0,6858368,0.005863,198018,4960,764.753906,783108,0,269.382812,275848,320196,0,99536,[],2023-02-05T20:58:11,"[susbrock, model_DB, ylfo6y2p]",None,0.0
89,b8s1tewx,mobilenetv1_0.25_96_c3_o2_l2,finished,5.596 M,https://wandb.ai/susbrock/model_DB/runs/b8s1tewx,mobilenetv1_0.25_96_c3_o2_l2,11.803 M,0.25,2,96,11803276,3,5595776,l2,0.005042,165346,11913876,850.914062,55296.0,5595776,0.005242,161410,3936,625.246094,640252,0,219.835938,225112,315428,0,9469

In [50]:
architectures = wandb_untrained_df["architecture"].unique().tolist()
architectures

['mobilenetv2_0.2_96_c3_o2_t3l128.MV1',
 'mobilenetv2_0.2_96_c3_o2_t3l256.MV1',
 'mobilenetv2_0.2_96_c3_o2_t3l512.MV1',
 'mobilenetv2_0.2_96_c3_o2_t3l1024.MV1',
 'mobilenetv2_0.2_96_c3_o2_t4l1024.MV1',
 'mobilenetv2_0.2_96_c3_o2_t4l512.MV1',
 'mobilenetv2_0.2_96_c3_o2_t4l256.MV1',
 'mobilenetv2_0.2_96_c3_o2_t4l128.MV1',
 'mobilenetv2_0.25_96_c3_o2_t4l256.MV1',
 'mobilenetv2_0.25_96_c3_o2_t4l512.MV1',
 'mobilenetv2_0.25_96_c3_o2_t5l512.MV1',
 'mobilenetv2_0.25_96_c3_o2_t5l256.MV1',
 'mobilenetv2_0.25_96_c3_o2_t6l128.MV1',
 'mobilenetv2_0.25_96_c3_o2_t6l256.MV1',
 'mobilenetv2_0.2_96_c3_o2_t6l512.MV1',
 'mobilenetv2_0.2_96_c3_o2_t5l128.MV1',
 'mobilenetv2_0.2_96_c3_o2_t5l256.MV1',
 'mobilenetv2_0.2_96_c3_o2_t5l512.MV1',
 'mobilenetv2_0.2_96_c3_o2_t5l1024.MV1',
 'mobilenetv2_0.2_96_c3_o2_t6l1024.MV1',
 'mobilenetv2_0.1_96_c3_o2_t6l512.MV1',
 'mobilenetv2_0.1_96_c3_o2_t6l256.MV1',
 'mobilenetv2_0.1_96_c3_o2_t6l128.MV1',
 'mobilenetv2_0.1_96_c3_o2_t3l1024.MV1',
 'mobilenetv2_0.1_96_c3_o2_t4

In [34]:
# training_csv_path = Path.cwd().joinpath("wand_training.csv")
# training_csv_path.is_file()

In [9]:
# training_df = pd.read_csv(training_csv_path)
# training_df.head()

In [51]:
training_df = training_df[training_df.state == "finished"]
training_df.head()

,id,run_name,state,seed,url,architecture,epochs,classes,dataset,momentum,optimizer,batch_size,learn_rate,batch_norm_momentum,hostname,tags,created_at,path,notes,test_accuracy
1,0400umu4,mobilenetv2_0.2_96_c3_o2_t4l256.MV1,finished,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t4l256.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T18:27:15,"[susbrock, model_DB_visual_wake_words, 0400umu4]",None,NaN
2,wh6k8s0w,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,finished,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T18:20:18,"[susbrock, model_DB_visual_wake_words, wh6k8s0w]",None,NaN
3,bq8c0lhi,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,finished,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T18:13:08,"[susbrock, model_DB_visual_wake_words, bq8c0lhi]",None,NaN
5,w4503fan,mobilenetv2_0.2_96_c3_o2_t5l1024.MV1,finished,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t5l1024.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,HP-Omen,[],2023-03-01T10:25:21,"[susbrock, model_DB_visual_wake_words, w4503fan]",None,0.822311
6,v2os0h5r,mobilenetv2_0.2_96_c3_o2_t5l128.MV1,finished,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t5l128.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,HP-Omen,[],2023-03-01T06:04:26,"[susbrock, model_DB_visual_wake_words, v2os0h5r]",None,0.817295


In [9]:
#architecture = "shufflenetv1_0.2_96_c3_o2_g4"

In [10]:
# test_accuracy = training_df.query(f"architecture=='{architecture}'")["test_accuracy"].values[0]
# test_loss = training_df.query(f"architecture=='{architecture}'")["test_loss"].values[0]
# print(test_accuracy) 
# print(test_loss)

0.8220377564430237
0.4569195806980133


In [12]:
# architectures =["shufflenetv2tiny_0.25_96_c3_o2_f24l512",
# "shufflenetv2tiny_0.25_96_c3_o2_f24l512",
# "shufflenetv2tiny_0.25_96_c3_o2_f24l128",
# "shufflenetv2tiny_0.25_96_c3_o2_f24l1024",
# "shufflenetv2tiny_0.1_96_c3_o2_f24l128",
# "shufflenetv2tiny_0.1_96_c3_o2_f24l1024",
# "shufflenetv2tiny_0.05_96_c3_o2_f24l128",
# "shufflenetv2tiny_0.05_96_c3_o2_f24l128"]


In [54]:
for architecture in architectures:
    test_accuracy = training_df.query(f"architecture=='{architecture}'")["test_accuracy"]#.values[0]
    #test_loss = training_df.query(f"architecture=='{architecture}'")["test_loss"]#.values[0]

    try:
        #run_id = df.query(f"Name=='{architecture}'")["id"]#.values[0]
        run_id = get_model_DB_run_id_from_architecture(architecture) # too slow, because of multiple API calls
        print(run_id)

 

        run = wandb.init(
        # Set the project where this run will be logged
        project="model_DB", 
        id = run_id, 
        resume="allow",
        )

        run.log({"test_accuracy" : test_accuracy,
                 "test_loss" : test_loss})

        wandb.finish()

    except:
        print(f"{architecture} has no run_id")

f7cli26n


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: susbrock. Use `wandb login --relogin` to force relogin


mobilenetv2_0.2_96_c3_o2_t3l128.MV1 has no run_id
le3i8f7h


allocate_tensors_ms_%,0.509
allocate_tensors_ms_avg,0.039
allocate_tensors_ms_first,0.039
arena_size,223712
first_inference_us,4502
inference_avg_us,543.907
init_us,148320
initialization_ms,148.32
model_size_MB,0.13398
modify_graph_with_delegate_mem_KB,892
modify_graph_with_delegate_ms_%,99.491


mobilenetv2_0.2_96_c3_o2_t3l256.MV1 has no run_id
phil00x9


...

mobilenetv1_0.25_96_c3_o2_l4 has no run_id
b8s1tewx


allocate_tensors_ms_%,36.019
allocate_tensors_ms_avg,0.076
allocate_tensors_ms_first,0.076
first_inference_us,3741
inference_avg_us,1594.5
init_us,122997
initialization_ms,122.997
model_size_MB,0.27585
modify_graph_with_delegate_mem_KB,0
modify_graph_with_delegate_ms_%,63.981
modify_graph_with_delegate_ms_avg,0.135


mobilenetv1_0.25_96_c3_o2_l2 has no run_id
ftt6trut


allocate_tensors_ms_%,36.53
allocate_tensors_ms_avg,0.08
allocate_tensors_ms_first,0.08
first_inference_us,6766
inference_avg_us,1064.44
init_us,134563
initialization_ms,134.563
model_size_MB,0.22511
modify_graph_with_delegate_mem_KB,0
modify_graph_with_delegate_ms_%,63.47
modify_graph_with_delegate_ms_avg,0.139


mobilenetv1_0.25_96_c3_o2_l1 has no run_id


In [61]:
untrained_models = get_untrained_models()
untrained_models

TypeError: '<' not supported between instances of 'str' and 'float'

In [58]:
def wandb_model_vww_training_DB():
    api = wandb.Api()
    runs = api.runs("susbrock/model_DB_visual_wake_words")

    config_df = pd.DataFrame()
    name = []
    tags =[]
    state = []
    url = []
    created_at = []
    path = []
    notes = []
    test_accuracies = []

    for run in runs:
        #print(run)
        name.append(run.name)
        tags.append(run.tags)
        state.append(run.state)
        url.append(run.url)
        created_at.append(run.created_at)
        path.append(run.path)
        notes.append(run.notes)
        test_accuracies.append(run.summary.get("test_accuracy"))

        df = pd.DataFrame([run.config], columns=run.config.keys())


        config_df = pd.concat([config_df, df], axis=0, ignore_index=True)
    col_names = ["run_name", "tags", "state", "url", "created_at", "path", "notes", "test_accuracy"]
    attrs_df = pd.DataFrame(zip(name, tags, state, url,created_at, path, notes, test_accuracies), columns=col_names)

    combined_df =  pd.concat([config_df, attrs_df], axis=1)

    combined_df.insert(1,"run_name" , combined_df.pop("run_name"))
    combined_df.insert(3,"architecture" , combined_df.pop("architecture"))   
    combined_df.insert(2,"state" , combined_df.pop("state")) 
    combined_df.insert(4,"url" , combined_df.pop("url")) 
    
    return combined_df

In [59]:
combined_df = wandb_model_vww_training_DB()
combined_df.head()

,id,run_name,state,seed,url,architecture,epochs,classes,dataset,momentum,optimizer,batch_size,learn_rate,batch_norm_momentum,hostname,tags,created_at,path,notes,test_accuracy
0,0k66mw8x,mobilenetv2_0.2_96_c3_o2_t3l256.MV1,running,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t3l256.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T19:56:28,"[susbrock, model_DB_visual_wake_words, 0k66mw8x]",None,NaN
1,tuq3r964,mobilenetv2_0.2_96_c3_o2_t4l256.MV1,running,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t4l256.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T19:09:57,"[susbrock, model_DB_visual_wake_words, tuq3r964]",None,NaN
2,0400umu4,mobilenetv2_0.2_96_c3_o2_t4l256.MV1,finished,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t4l256.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T18:27:15,"[susbrock, model_DB_visual_wake_words, 0400umu4]",None,NaN
3,wh6k8s0w,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,finished,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T18:20:18,"[susbrock, model_DB_visual_wake_words, wh6k8s0w]",None,NaN
4,bq8c0lhi,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,finished,15,https://wandb.ai/susbrock/model_DB_visual_wake...,mobilenetv2_0.2_96_c3_o2_t4l512.MV1,50,2,vvw_minval_datagen_fix,0.9,Adam,50,0.001,0.9,NaN,[],2023-03-01T18:13:08,"[susbrock, model_DB_visual_wake_words, bq8c0lhi]",None,NaN


In [ ]:
# api = wandb.Api()

# # Project is specified by <entity/project-name>
# #runs = api.runs("susbrock/mobilenetv1vvw")
# runs = api.runs("susbrock/model_DB_visual_wake_words")

# summary_list, config_list, name_list = [], [], []
# for run in runs: 
#     # .summary contains the output keys/values for metrics like accuracy.
#     #  We call ._json_dict to omit large files 
#     summary_list.append(run.summary._json_dict)

#     # .config contains the hyperparameters.
#     #  We remove special values that start with _.
#     config_list.append(
#         {k: v for k,v in run.config.items()
#           if not k.startswith('_')})

#     # .name is the human-readable name of the run.
#     name_list.append(run.name)

# runs_df = pd.DataFrame({
#     "summary": summary_list,
#     "config": config_list,
#     "name": name_list
#     })


# Additional data munging